In [1]:
import pandas as pd
import re

In [2]:
def _parse_line(line,rx_dict):
    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """

    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

In [3]:
re_dia = r'Dia #(?P<dia>\d+)\n'
re_panico = r'Panico: (?P<panico>.+)\n'
re_hab = r'Numero de habitantes: (?P<habitantes>\d+)\n'
re_muertos = r'Numero de defunciones: (?P<muertos>\d+)\n'
re_personas_infectadas = r'Numero de personas infectadas: (?P<infectados>\d+)\n'
re_personas_hosp = r'Numero personas hospitalizadas: (?P<hospitalizados>\d+)\n'
re_sepa_mayor = r'Sepa de mayor contagio: (?P<sepaMayor>\d+)\n'
re_tot_moyotes = r'Poblacion de mosquitos: (?P<numMoyotes>\d+)\n'
re_moyotes_inf = r'Mosquitos infectados: (?P<numMoyotesInfectados>\d+)\n'

re_triger_urbe = r'METRICAS DE LA URBE:'
re_triger_area = r'INFORMACION DEL AREA (?P<numArea>\d+)\n'
rx_dia_dict = {
    'dia':re.compile(re_dia),
    'panico':re.compile(re_panico),
    'habitantes': re.compile(re_hab),
    'muertos': re.compile(re_muertos),
    'infectados': re.compile(re_personas_infectadas),
    'hospitalizados':re.compile(re_personas_hosp),
    'sepaMayor': re.compile(re_sepa_mayor),
    'numMoyotes': re.compile(re_tot_moyotes),
    'numMoyotesInfectados': re.compile(re_moyotes_inf),
    'triger_urbe':re.compile(re_triger_urbe),
    'triger_area':re.compile(re_triger_area)
    
}
def parse_file(filepath):
    data = []
    area = {}
    with open(filepath,'r') as file_object:
        line = file_object.readline()
        while line:
            key, match = _parse_line(line,rx_dia_dict)
            
            if(key or match):
                if(key == 'triger_area'):
                    nomAtt = 'Area_'+ str(match.group('numArea'))
                    area[nomAtt] = parse_area(file_object)
                elif(key == 'triger_urbe'): 
                    area.update(parse_urbe(file_object))
                else:
                    if(key == 'panico'):
                        area.update({key:float(match.group(str(key)))})
                    else:
                        area.update({key:int(match.group(str(key)))})
            line = file_object.readline()
    return area

In [4]:
re_moyotes_urbe = r'Total de mosquitos: (?P<moyotes_urbe>\d+)\n'
re_moyotes_sanos_urbe = r'   Sanos: (?P<moyotes_sanos_urbe>\d+)\n'

re_personas_urbe = r'Total de Personas: (?P<personas_urbe>\d+)\n'
re_personas_sanos_urbe = r'   Sanas: (?P<personas_sanos_urbe>\d+)\n'

re_urbe_sailr = r'Metricas Areas:'

rx_urbe_dict = {
    'moyotes_urbe':re.compile(re_moyotes_urbe),
    'moyotes_sanos_urbe':re.compile(re_moyotes_sanos_urbe),
    'personas_urbe':re.compile(re_personas_urbe),
    'personas_sanos_urbe':re.compile(re_personas_sanos_urbe),
    'triger_salir':re.compile(re_urbe_sailr)
}
def parse_urbe(file_object):
    line = file_object.readline()
    metricas = {}
    key = ''
    while key != 'triger_salir':
        key,match = _parse_line(line,rx_urbe_dict)
        if(key == 'moyotes_sanos_urbe'):
            metricas.update({key:int(match.group(str(key)))})
            metricas['moyotes_infectados'] = parse_serotipos(file_object)
        elif(key == 'personas_sanos_urbe'):
            metricas.update({key:match.group(str(key))})
            metricas['personas_infectadas'] = parse_serotipos(file_object)
        elif(key == 'triger_salir'):
            break
        elif(key):
            metricas.update({key:int(match.group(str(key)))})
        line = file_object.readline()
    return metricas

In [5]:
re_se_1 = r'(   Con serotipo 1: | Serotipo 1: )(?P<sepa_1>\d+)\n'
re_se_2 = r'(   Con serotipo 2: | Serotipo 2: )(?P<sepa_2>\d+)\n'
re_se_3 = r'(   Con serotipo 3: | Serotipo 3: )(?P<sepa_3>\d+)\n'
re_se_4 = r'(   Con serotipo 4: | Serotipo 4: )(?P<sepa_4>\d+)\n'
rx_sepas_dict = {    
    'sepa_1':re.compile(re_se_1),
    'sepa_2':re.compile(re_se_2),
    'sepa_3':re.compile(re_se_3),
    'sepa_4':re.compile(re_se_4),
}
def parse_serotipos(file_object):
    line = file_object.readline()
    sepas = {}
    for i in range(4):
        key, match = _parse_line(line,rx_sepas_dict)
        sepas.update({key:int(match.group(str(key)))})
        line = file_object.readline()
    return sepas
        

In [6]:

re_area_folio = r'INFORMACION DEL AREA (?P<folio>\d+)\n'
re_area_tipo = r'Es principalmente (?P<tipo>.+)\n'
re_area_pob = r'Residentes del area: (?P<poblacion>\d+)\n'
re_area_trab = r'Trabajan en el area: (?P<trabajadores>\d+)\n'
re_area_recs = r'Se recrean en el area: (?P<recreadores>\d+)\n'
re_area_personas_infectadas_tot = r'Infectados totales: (?P<personas_infectadas>\d+)\n'
re_area_personas_infectadas_vivas = r'Infectados vivos totales: (?P<personas_infectadas_vivas>\d+)\n'
re_area_moyotes = r'Totales: (?P<moyotes>\d+)\n'
re_area_moyotes_sanos = r'   Sanos: (?P<moyotes_sanos>\d+)\n'
re_area_muertes = r'Fallecidos totales: (?P<muertes>\d+)\n'
re_area_salir = r'fin de la info del area'
rx_area_dict = {
    'folio':re.compile(re_area_folio),
    'tipo':re.compile(re_area_tipo),
    'poblacion':re.compile(re_area_pob),
    'trabajadores':re.compile(re_area_trab),
    'recreadores':re.compile(re_area_recs),
    'personas_infectadas':re.compile(re_area_personas_infectadas_tot),
    'personas_infectadas_vivas':re.compile(re_area_personas_infectadas_vivas),
    'moyotes':re.compile(re_area_moyotes),
    'moyotes_sanos':re.compile(re_area_moyotes_sanos),
    'muertes':re.compile(re_area_muertes),
    'triger_salir':re.compile(re_area_salir)
}
def parse_area(file_object):
    line = file_object.readline()
    info_area = {}
    while(True):
        key,match = _parse_line(line,rx_area_dict)
        if(key):
            if(key == 'triger_salir'): 
                break
            if(key == 'tipo'):
                info_area.update({key:match.group(str(key))})
            else:
                info_area.update({key:int(match.group(str(key)))})
            if(key == 'moyotes'):
                line = file_object.readline()
                key,match = _parse_line(line, rx_area_dict)
                info_area['moyotes_infectados_por_sepa'] = parse_serotipos(file_object)
            elif(key not in ['tipo','poblacion','trabajadores','recreadores']):
                nomAtt = str(key) + '_por_sepa'
                info_area[nomAtt] = parse_serotipos(file_object)
            
        line = file_object.readline()
    return info_area

In [8]:
import os
def read_all_data(folio_sim):
    path = '../sim'
    archivos = []
    for r,d,files in os.walk(path):
        for file in files:
            start = str(folio_sim) + '_'
            if start in file:
                archivos.append(file)
    data = []
    
    for archivo in archivos:
        dict_info = parse_file('../sim/'+str(archivo))
        #flatten all areas
        for i in range(3):
            pass
        #flatten metricas
        
        data.append(dict_info)
    data = pd.DataFrame(data).sort_values('dia').replace('true',1).replace('false',0).replace('null',-1)
    return data 

In [49]:
# a = parse_file('../sim/0_0')

In [18]:
# a.Area_1[0]#['tipo']

{'tipo': 'RESIDENCIAL',
 'poblacion': 637,
 'trabajadores': 0,
 'recreadores': 0,
 'personas_infectadas': 185,
 'personas_infectadas_por_sepa': {'sepa_1': 149,
  'sepa_2': 36,
  'sepa_3': 0,
  'sepa_4': 0},
 'personas_infectadas_vivas': 183,
 'personas_infectadas_vivas_por_sepa': {'sepa_1': 148,
  'sepa_2': 35,
  'sepa_3': 0,
  'sepa_4': 0},
 'moyotes': 287,
 'moyotes_infectados_por_sepa': {'sepa_1': 0,
  'sepa_2': 0,
  'sepa_3': 0,
  'sepa_4': 0},
 'muertes': 2,
 'muertes_por_sepa': {'sepa_1': 1, 'sepa_2': 1, 'sepa_3': 0, 'sepa_4': 0}}

In [17]:
# a.head()

,Area_1,Area_2,Area_3,dia,habitantes,hospitalizados,infectados,moyotes_infectados,moyotes_sanos_urbe,moyotes_urbe,muertos,numMoyotes,numMoyotesInfectados,panico,personas_infectadas,personas_sanos_urbe,personas_urbe,sepaMayor
116,"{'tipo': 'RESIDENCIAL', 'poblacion': 800, 'tra...","{'tipo': 'LABORAL', 'poblacion': 0, 'trabajado...","{'tipo': 'RECREATIVA', 'poblacion': 0, 'trabaj...",0,800,0,0,"{'sepa_1': 78, 'sepa_2': 61, 'sepa_3': 0, 'sep...",886,1025,0,1100,214,0.0,"{'sepa_1': 0, 'sepa_2': 0, 'sepa_3': 0, 'sepa_...",0,800,1
84,"{'tipo': 'RESIDENCIAL', 'poblacion': 800, 'tra...","{'tipo': 'LABORAL', 'poblacion': 0, 'trabajado...","{'tipo': 'RECREATIVA', 'poblacion': 0, 'trabaj...",1,800,0,158,"{'sepa_1': 78, 'sepa_2': 61, 'sepa_3': 0, 'sep...",886,1025,0,1100,214,0.0,"{'sepa_1': 86, 'sepa_2': 72, 'sepa_3': 0, 'sep...",0,800,1
18,"{'tipo': 'RESIDENCIAL', 'poblacion': 800, 'tra...","{'tipo': 'LABORAL', 'poblacion': 0, 'trabajado...","{'tipo': 'RECREATIVA', 'poblacion': 0, 'trabaj...",2,800,0,184,"{'sepa_1': 78, 'sepa_2': 61, 'sepa_3': 0, 'sep...",886,1025,0,1100,214,0.0,"{'sepa_1': 98, 'sepa_2': 86, 'sepa_3': 0, 'sep...",0,800,1
123,"{'tipo': 'RESIDENCIAL', 'poblacion': 800, 'tra...","{'tipo': 'LABORAL', 'poblacion': 0, 'trabajado...","{'tipo': 'RECREATIVA', 'poblacion': 0, 'trabaj...",3,800,0,187,"{'sepa_1': 78, 'sepa_2': 61, 'sepa_3': 0, 'sep...",886,1025,0,1100,214,0.0,"{'sepa_1': 100, 'sepa_2': 87, 'sepa_3': 0, 'se...",0,800,1
96,"{'tipo': 'RESIDENCIAL', 'poblacion': 800, 'tra...","{'tipo': 'LABORAL', 'poblacion': 0, 'trabajado...","{'tipo': 'RECREATIVA', 'poblacion': 0, 'trabaj...",4,800,0,187,"{'sepa_1': 78, 'sepa_2': 61, 'sepa_3': 0, 'sep...",886,1025,0,1100,214,0.0,"{'sepa_1': 100, 'sepa_2': 87, 'sepa_3': 0, 'se...",0,800,1
